In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
# Plotly library
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)


from keras import callbacks
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam


pd.set_option('display.max_columns', None)

In [ ]:
df_train = pd.read_csv("/kaggle/input/microsoft-malware-prediction/train.csv")
#sampling
sample=100000
df_train = df_train.sample(sample)
# x=gc.collect()

In [ ]:
df_train

In [ ]:
df_test = pd.read_csv("/kaggle/input/microsoft-malware-prediction/test.csv")
#sampling
sample=50000
df_test = df_test.sample(sample)
df_test

In [ ]:
df_train.columns

In [ ]:
df_train.info()

In [ ]:
#missing value chart
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
#drop variables with missing values >=20% in the train dataframe
i=0
for col in df_train.columns:
    if (df_train[col].isnull().sum()/len(df_train[col])*100) >=10:
        print("Dropping column", col)
        df_train.drop(labels=col,axis=1,inplace=True)
        i=i+1
        
print("Total number of columns dropped in train dataframe", i)

In [ ]:
 df_train.drop(columns='SMode',inplace=True)
    

In [ ]:
#drop variables with missing values >=20% in the train dataframe
i=0
for col in df_test.columns:
    if (df_test[col].isnull().sum()/len(df_test[col])*100) >=10:
        print("Dropping column", col)
        df_test.drop(labels=col,axis=1,inplace=True)
        i=i+1
        
print("Total number of columns dropped in train dataframe", i)

In [ ]:
# Categorize attribute(column) based on its type
dtypes = df_train.dtypes
numerics = [ 'int64', 'float64']
num_columns = [c for c,v in dtypes.items() if v in numerics]
cat_columns = [c for c,v in dtypes.items() if v not in numerics]

print("Numerical Column : ",len(num_columns),"\n")
print(num_columns)
print("\n\nCategorical Column",len(cat_columns),"\n")
print(cat_columns)

In [ ]:
# impute missing value with mean and mode

def replace_missing_data(df, mis_vars):
    print('##### Replacing missing values with mode of features #####')
    for var in mis_vars:
        if var in cat_columns:
            df[var] = df[var].fillna(df[var].mode()[0])
        else:
            df[var] = df[var].fillna(df[var].mean())
    return df

In [ ]:
mis_var = [var for var in df_train.columns if df_train[var].isnull().sum() > 0]
df_train = replace_missing_data(df_train, mis_var)

In [ ]:
mis_var = [var for var in df_test.columns if df_test[var].isnull().sum() > 0]
df_test = replace_missing_data(df_test, mis_var)

In [ ]:
#missing value chart
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percent'])
missing_data.head(3)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_train.drop(columns=['MachineIdentifier'],inplace=True)
df_test.drop(columns=['MachineIdentifier'],inplace=True)  

In [ ]:
df_train.shape, df_test.shape

In [ ]:
corrmat = df_train.corr().abs()
f, ax = plt.subplots(figsize=(30, 30))
cols = corrmat.nlargest(50, 'HasDetections')['HasDetections'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.2)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10},yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
sns.countplot(x=df_train['HasDetections'])

In [ ]:
plt.figure(figsize=(20,20))
for i in range(len(cols[:25])):
    plt.subplot(5,5, i+1)
    sns.scatterplot(x=df_train[cols[i]], y=df_train['HasDetections'])
plt.suptitle('Relation between HasDetections and 25 important features')

In [ ]:
plt.figure(figsize=(20,20))
for i in range(len(cols[:25])):
    plt.subplot(5,5, i+1) 
    sns.boxplot(x=df_train[cols[i]])
plt.suptitle('Values for 25 important features')

In [ ]:
df_unique = df_train.nunique().to_frame().reset_index()
df_unique.columns = ['Variable','DistinctCount']
# df_unique.head(50)

In [ ]:
# df_unique.loc[(df_unique['DistinctCount']<10 )& (df_unique['Variable'].apply(lambda x: x in cat_columns))]

In [ ]:
df_unique.loc[df_unique['Variable'].apply(lambda x: x in cat_columns)]

In [ ]:
# print(df_train['ProductName'].unique())
print(df_train['ProductName'].value_counts())
print((df_train['ProductName'].value_counts()*100)/2000000)
# this plot shows that the data variation in this column is skewed. this column must be dropped.

In [ ]:
print((df_train['EngineVersion'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['EngineVersion'].value_counts(normalize=True)
mask = value_counts < 0.1
df_train.loc[df_train['EngineVersion'].isin(value_counts[mask].index), 'EngineVersion'] = 'Other'
df_test.loc[df_test['EngineVersion'].isin(value_counts[mask].index), 'EngineVersion'] = 'Other'
print((df_train['EngineVersion'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('EngineVersion').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['AppVersion'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['AppVersion'].value_counts(normalize=True)
mask = value_counts < 0.1
df_train.loc[df_train['AppVersion'].isin(value_counts[mask].index), 'AppVersion'] = 'Other'
df_test.loc[df_test['AppVersion'].isin(value_counts[mask].index), 'AppVersion'] = 'Other'
print((df_train['AppVersion'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('AppVersion').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['AvSigVersion'].value_counts()*100)/2000000)
#data shows that 8k different value and its string data
# dont know may be delete this

In [ ]:
print((df_train['Platform'].value_counts()*100)/2000000)
#data is skewed must delete

In [ ]:
print((df_train['Processor'].value_counts()*100)/2000000)
#data is skewed must delete

In [ ]:
print((df_train['OsVer'].value_counts()*100)/2000000)
#data is skewed must delete

In [ ]:
print((df_train['OsPlatformSubRelease'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['OsPlatformSubRelease'].value_counts(normalize=True)
mask = value_counts < 0.05
df_train.loc[df_train['OsPlatformSubRelease'].isin(value_counts[mask].index), 'OsPlatformSubRelease'] = 'Other'
df_test.loc[df_test['OsPlatformSubRelease'].isin(value_counts[mask].index), 'OsPlatformSubRelease'] = 'Other'

print((df_train['OsPlatformSubRelease'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('OsPlatformSubRelease').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['OsBuildLab'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['OsBuildLab'].value_counts(normalize=True)
mask = value_counts < 0.05
df_train.loc[df_train['OsBuildLab'].isin(value_counts[mask].index), 'OsBuildLab'] = 'Other'
df_test.loc[df_test['OsBuildLab'].isin(value_counts[mask].index), 'OsBuildLab'] = 'Other'
print((df_train['OsBuildLab'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('OsBuildLab').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['SkuEdition'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['SkuEdition'].value_counts(normalize=True)
mask = value_counts < 0.1
df_train.loc[df_train['SkuEdition'].isin(value_counts[mask].index), 'SkuEdition'] = 'Other'
df_test.loc[df_test['SkuEdition'].isin(value_counts[mask].index), 'SkuEdition'] = 'Other'

print((df_train['SkuEdition'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('SkuEdition').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_MDC2FormFactor'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['Census_MDC2FormFactor'].value_counts(normalize=True)
mask = value_counts < 0.1
df_train.loc[df_train['Census_MDC2FormFactor'].isin(value_counts[mask].index), 'Census_MDC2FormFactor'] = 'Other'
df_test.loc[df_test['Census_MDC2FormFactor'].isin(value_counts[mask].index), 'Census_MDC2FormFactor'] = 'Other'
print((df_train['Census_MDC2FormFactor'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_MDC2FormFactor').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_DeviceFamily'].value_counts()*100)/2000000)

#data is skewed must delete the column

In [ ]:
print((df_train['Census_PrimaryDiskTypeName'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['Census_PrimaryDiskTypeName'].value_counts(normalize=True)
mask = value_counts < 0.1
df_train.loc[df_train['Census_PrimaryDiskTypeName'].isin(value_counts[mask].index), 'Census_PrimaryDiskTypeName'] = 'UNKNOWN'
df_test.loc[df_test['Census_PrimaryDiskTypeName'].isin(value_counts[mask].index), 'Census_PrimaryDiskTypeName'] = 'UNKNOWN'
print((df_train['Census_PrimaryDiskTypeName'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_PrimaryDiskTypeName').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_ChassisTypeName'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 1% to other
value_counts = df_train['Census_ChassisTypeName'].value_counts(normalize=True)
mask = value_counts < 0.01
df_train.loc[df_train['Census_ChassisTypeName'].isin(value_counts[mask].index), 'Census_ChassisTypeName'] = 'Other'
df_test.loc[df_test['Census_ChassisTypeName'].isin(value_counts[mask].index), 'Census_ChassisTypeName'] = 'Other'

print((df_train['Census_ChassisTypeName'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_ChassisTypeName').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_PowerPlatformRoleName'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['Census_PowerPlatformRoleName'].value_counts(normalize=True)
mask = value_counts < 0.1
df_train.loc[df_train['Census_PowerPlatformRoleName'].isin(value_counts[mask].index), 'Census_PowerPlatformRoleName'] = 'Other'
df_test.loc[df_test['Census_PowerPlatformRoleName'].isin(value_counts[mask].index), 'Census_PowerPlatformRoleName'] = 'Other'

print((df_train['Census_PowerPlatformRoleName'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_PowerPlatformRoleName').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_OSVersion'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 3% to other
value_counts = df_train['Census_OSVersion'].value_counts(normalize=True)
mask = value_counts < 0.03
df_train.loc[df_train['Census_OSVersion'].isin(value_counts[mask].index), 'Census_OSVersion'] = 'Other'
print((df_train['Census_OSVersion'].value_counts()*100)/2000000)

# too much variation for categorical value must delete

In [ ]:
print((df_train['Census_OSArchitecture'].value_counts()*100)/2000000)
#data is skewed must delete

In [ ]:
print((df_train['Census_OSBranch'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 5% to other
value_counts = df_train['Census_OSBranch'].value_counts(normalize=True)
mask = value_counts < 0.05
df_train.loc[df_train['Census_OSBranch'].isin(value_counts[mask].index), 'Census_OSBranch'] = 'Other'
df_test.loc[df_test['Census_OSBranch'].isin(value_counts[mask].index), 'Census_OSBranch'] = 'Other'

print((df_train['Census_OSBranch'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_OSBranch').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_OSEdition'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['Census_OSEdition'].value_counts(normalize=True)
mask = value_counts < 0.10
df_train.loc[df_train['Census_OSEdition'].isin(value_counts[mask].index), 'Census_OSEdition'] = 'Other'
df_test.loc[df_test['Census_OSEdition'].isin(value_counts[mask].index), 'Census_OSEdition'] = 'Other'
print((df_train['Census_OSEdition'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_OSEdition').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_OSSkuName'].value_counts()*100)/2000000)

In [ ]:
#change the value which is less tha 10% to other
value_counts = df_train['Census_OSSkuName'].value_counts(normalize=True)
mask = value_counts < 0.10
df_train.loc[df_train['Census_OSSkuName'].isin(value_counts[mask].index), 'Census_OSSkuName'] = 'Other'
df_test.loc[df_test['Census_OSSkuName'].isin(value_counts[mask].index), 'Census_OSSkuName'] = 'Other'

print((df_train['Census_OSSkuName'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_OSSkuName').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_OSInstallTypeName'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_OSInstallTypeName').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_OSWUAutoUpdateOptionsName'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_OSWUAutoUpdateOptionsName').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_GenuineStateName'].value_counts()*100)/2000000)
#data is skewed must delete

In [ ]:
print((df_train['Census_ActivationChannel'].value_counts()*100)/2000000)

In [ ]:
df_train.groupby('Census_ActivationChannel').size().plot(kind='pie', autopct='%.2f')

In [ ]:
print((df_train['Census_FlightRing'].value_counts()*100)/2000000)

# data is skewed must delete

In [ ]:
# ProductName,Processor, OsVer, Census_DeviceFamily, Census_OSArchitecture, Census_GenuineStateName, Census_FlightRing
# all of these columns data is skewed.
# must delete this
# "AvSigVersion","Census_OSVersion"
df_train.drop(columns=['ProductName','Processor', 'OsVer', 'Census_DeviceFamily', 'Census_OSArchitecture', 'Census_GenuineStateName', 'Census_FlightRing'],inplace=True)
df_test.drop(columns=['ProductName','Processor', 'OsVer', 'Census_DeviceFamily', 'Census_OSArchitecture', 'Census_GenuineStateName', 'Census_FlightRing'],inplace=True) 

In [ ]:

# all of these columns have too many data for a categorical type
# must delete this
df_train.drop(columns=['AvSigVersion','Census_OSVersion'],inplace=True)
df_test.drop(columns=['AvSigVersion','Census_OSVersion',],inplace=True)

In [ ]:
df_train.info()

one hot encoding

In [ ]:
dtypes = df_train.dtypes
numerics = [ 'int64', 'float64']
num_columns = [c for c,v in dtypes.items() if v in numerics]
cat_columns = [c for c,v in dtypes.items() if v not in numerics]

print("Numerical Column : ",len(num_columns),"\n")
print(num_columns)
print("\n\nCategorical Column",len(cat_columns),"\n")
print(cat_columns)


In [ ]:
one_hot_encoded_data = pd.get_dummies(df_train, columns = ['EngineVersion', 'AppVersion', 'Platform', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'Census_MDC2FormFactor', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_ActivationChannel'])
one_hot_encoded_test_data = pd.get_dummies(df_test, columns = ['EngineVersion', 'AppVersion', 'Platform', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'Census_MDC2FormFactor', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_ActivationChannel'])

one_hot_encoded_data

Model Traning

In [ ]:
# from keras import callbacks
# from sklearn.metrics import roc_auc_score

   

class printAUC(callbacks.Callback):
    def __init__(self, X_train, y_train, validation_data):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = y_train
        self.validation_data = validation_data
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Train AUC: " + str(auc))
        pred = self.model.predict(self.validation_data[0])
        auc = roc_auc_score(self.validation_data[1], pred)
        print ("Validation AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
        return



In [ ]:
# from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization, Activation
# from keras.callbacks import LearningRateScheduler
# from keras.optimizers import Adam

In [ ]:
#SPLIT TRAIN AND VALIDATION SET
X=one_hot_encoded_data.loc[:, one_hot_encoded_data.columns != 'HasDetections']
y= one_hot_encoded_data['HasDetections']

In [ ]:
one_hot_encoded_data.shape,X.shape, y.shape

In [ ]:
X

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3,random_state=4)

In [ ]:
X_train.shape, y_train.shape,X_val.shape

In [ ]:
len(X_train.columns)

In [ ]:
# BUILD MODEL
model = Sequential()
model.add(Dense(200,input_dim=len(X_train.columns)))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(400))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate=0.01), loss="binary_crossentropy", metrics=["accuracy"])
annealer = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** x)


In [ ]:
# # TRAIN MODEL

history = model.fit(X_train, y_train, batch_size=64, epochs=20, callbacks=[annealer, printAUC(X_train, y_train, (X_val, y_val))], validation_data=(X_val, y_val), verbose=2)


In [ ]:
#accuracy
plt.figure(0)
plt.plot(history.history['accuracy'],label='tranning accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show

In [ ]:
plt.plot(history.history['loss'],label='tranning loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show